## Import

In [1]:
import pandas as pd

## Reading In and Reviewing Data Files

In [2]:
df_tsc_price_change = pd.read_csv('data/TSC_Price_Changes.csv')

In [3]:
df_tsc_price_change.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3886 entries, 0 to 3885
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   SCRUB_ITEM        3886 non-null   int64  
 1   PRICE_START_DATE  3886 non-null   object 
 2   PRICE_END_DATE    3886 non-null   object 
 3   TSC_PRICE         3886 non-null   float64
dtypes: float64(1), int64(1), object(2)
memory usage: 121.6+ KB


In [4]:
df_tsc_price_change.head()

,SCRUB_ITEM,PRICE_START_DATE,PRICE_END_DATE,TSC_PRICE
0,1,2019-08-26,2019-10-03,199.99
1,1,2019-10-04,2019-10-08,179.99
2,1,2019-10-09,2019-10-27,199.99
3,1,2019-10-28,2019-10-29,179.99
4,1,2019-10-30,2019-11-05,199.99


In [5]:
df_comp_price_change = pd.read_csv('data/Competitors_Price_Changes.csv')

In [6]:
df_comp_price_change.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90953 entries, 0 to 90952
Data columns (total 8 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   SCAN_DATE_TIME               90953 non-null  object 
 1   SCRUB_ITEM                   90953 non-null  int64  
 2   SCRUB_COMPETITOR             90953 non-null  int64  
 3   TSC_PRICE                    90953 non-null  float64
 4   COMPETITOR_PRICE             90953 non-null  float64
 5   CPI_COMPETITIVE_PRICE_INDEX  90953 non-null  float64
 6   COMPETITOR_SHIPPING          51396 non-null  float64
 7   COMPETITOR_REG_PRICE         90953 non-null  float64
dtypes: float64(5), int64(2), object(1)
memory usage: 5.6+ MB


In [7]:
df_comp_price_change.head()

,SCAN_DATE_TIME,SCRUB_ITEM,SCRUB_COMPETITOR,TSC_PRICE,COMPETITOR_PRICE,CPI_COMPETITIVE_PRICE_INDEX,COMPETITOR_SHIPPING,COMPETITOR_REG_PRICE
0,2020-08-06 04:20:00.000,1,1,199.99,205.58,102.7951,0.0,205.58
1,2020-08-06 04:21:00.000,1,2,199.99,205.58,102.7951,0.0,205.58
2,2020-08-07 04:24:00.000,1,1,199.99,205.58,102.7951,0.0,205.58
3,2020-08-07 04:24:00.000,1,2,199.99,205.58,102.7951,0.0,205.58
4,2020-08-08 04:22:00.000,1,1,199.99,205.58,102.7951,0.0,205.58


In [8]:
df_tsc_sales = pd.read_csv('data/TSC_Sales_Data.csv')

In [9]:
df_tsc_sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70829 entries, 0 to 70828
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   SCRUB_ITEM      70829 non-null  int64  
 1   TIME_DIM_KEY    70829 non-null  int64  
 2   SCRUB_TRANS_NO  70829 non-null  int64  
 3   TRANS_SEQ_NO    70829 non-null  int64  
 4   UNIT_QTY        70829 non-null  float64
 5   UNIT_PRICE      70829 non-null  float64
 6   UNIT_COST       70829 non-null  float64
dtypes: float64(3), int64(4)
memory usage: 3.8 MB


In [10]:
df_tsc_sales.head()

,SCRUB_ITEM,TIME_DIM_KEY,SCRUB_TRANS_NO,TRANS_SEQ_NO,UNIT_QTY,UNIT_PRICE,UNIT_COST
0,1,20180402,1296,2,2.0,322.99,132.97
1,1,20181206,6878,1,1.0,322.99,132.97
2,1,20181210,7010,2,2.0,322.99,132.97
3,1,20190214,8380,2,1.0,322.99,132.97
4,1,20190502,10639,1,2.0,219.99,132.97


## Converting to Date and Time

In [11]:
df_tsc_price_change['PRICE_START_DATE'] = pd.to_datetime(df_tsc_price_change['PRICE_START_DATE'],
                                                         format='%Y-%m-%d')

In [12]:
df_tsc_price_change['PRICE_END_DATE'] = pd.to_datetime(df_tsc_price_change['PRICE_END_DATE'],
                                                       errors='coerce', #convert errors for nulls
                                                       format='%Y-%m-%d')

Some of the end dates have errors because the year is 9999 for items that do not have a price_end_date. Therefore, when we convert the data to date and time, we used 'errors=coerce', converting errors to nulls.

In [13]:
df_tsc_price_change.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3886 entries, 0 to 3885
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   SCRUB_ITEM        3886 non-null   int64         
 1   PRICE_START_DATE  3886 non-null   datetime64[ns]
 2   PRICE_END_DATE    3815 non-null   datetime64[ns]
 3   TSC_PRICE         3886 non-null   float64       
dtypes: datetime64[ns](2), float64(1), int64(1)
memory usage: 121.6 KB


In [14]:
#getting the first 10 characters of the date to get the Y-m-d
df_comp_price_change['SCAN_DATE_TIME'] = df_comp_price_change['SCAN_DATE_TIME'].str[:10]

In [15]:
df_comp_price_change['SCAN_DATE_TIME'] = pd.to_datetime(df_comp_price_change['SCAN_DATE_TIME'],format='%Y-%m-%d')

In [16]:
df_tsc_sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70829 entries, 0 to 70828
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   SCRUB_ITEM      70829 non-null  int64  
 1   TIME_DIM_KEY    70829 non-null  int64  
 2   SCRUB_TRANS_NO  70829 non-null  int64  
 3   TRANS_SEQ_NO    70829 non-null  int64  
 4   UNIT_QTY        70829 non-null  float64
 5   UNIT_PRICE      70829 non-null  float64
 6   UNIT_COST       70829 non-null  float64
dtypes: float64(3), int64(4)
memory usage: 3.8 MB


In [17]:
df_tsc_sales['TIME_DIM_KEY'] = pd.to_datetime(df_tsc_sales['TIME_DIM_KEY'],format='%Y%m%d')

In [19]:
df_tsc_sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70829 entries, 0 to 70828
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   SCRUB_ITEM      70829 non-null  int64         
 1   TIME_DIM_KEY    70829 non-null  datetime64[ns]
 2   SCRUB_TRANS_NO  70829 non-null  int64         
 3   TRANS_SEQ_NO    70829 non-null  int64         
 4   UNIT_QTY        70829 non-null  float64       
 5   UNIT_PRICE      70829 non-null  float64       
 6   UNIT_COST       70829 non-null  float64       
dtypes: datetime64[ns](1), float64(3), int64(3)
memory usage: 3.8 MB
